<a href="https://colab.research.google.com/github/Federico6419/MachineLearningProject/blob/main/MachineLearningProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install libraries


In [1]:
!pip install gymnasium              #Install Gymnasium
!pip install swig                   #This solves the error in the installation of gymnasium[box2d]
!pip install gymnasium[box2d]       #Install Box2D
!pip install gym-notebook-wrapper   #This installs Gym-Notebook-Wrapper, that provides small wrappers for running and rendering OpenAI Gym

#To solve the xvfb missing file problem
!sudo apt-get install xvfb
!pip install xvfbwrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2373073 sha256=03bddad5a3ecc7b7594f5acab7d67482779ec72cd999842de68d2e296481bf55
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libx

## Import libraries

In [2]:
!git clone https://github.com/Federico6419/MachineLearningProject               #It clones my github repository
%cd MachineLearningProject

import gymnasium as gym
import gnwrapper
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import cv2
import config
from model import Model
from collections import deque
from torchvision import transforms
import base64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")           #Select the Device

Cloning into 'MachineLearningProject'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 323 (delta 121), reused 139 (delta 89), pack-reused 130
Receiving objects: 100% (323/323), 2.24 MiB | 29.72 MiB/s, done.
Resolving deltas: 100% (193/193), done.
/content/MachineLearningProject


#Training

In [ ]:
#Manage Google Drive
from google.colab import drive
drive.mount('/content/drive')

#Initialize the Variables
episode_reward = 0                                 #This is the Variable in which it will be stored the Reward of each Episode
tot_negative_reward = 0                            #This Variable counts the number of consecutive Negative Rewards
time_frame_counter = 1                             #This is the Counter of the Frames
buffer = deque([], config.BUFFER_SIZE)             #Initialize the Queue that contains the past experience
epsilon = config.MAX_EPSILON
if(config.LOAD_CHECKPOINT):                        #If Checkpoints are loaded, initialize Epsilon to a specific value
    epsilon = config.LOADED_EPSILON

alpha = config.ALPHA
decay = config.EPSILON_DECAY

#Array that wikk store the Rewards
cum_reward_table = np.zeros(config.NUM_EPISODES)
cum_reward_nn = np.zeros(config.NUM_EPISODES)

#Initialize the Model
model = Model().to(config.DEVICE)
optimizer = optim.Adam(model.parameters(), lr=config.LR)

#Initialize the Target Model
target_model = Model().to(config.DEVICE)
optimizer_target = optim.Adam(target_model.parameters(), lr=config.LR)

#If Checkpoints are loaded, load them on the Models
if(config.LOAD_CHECKPOINT):
    config.load_model(config.LOAD_CHECKPOINT_FOLDER,model,optimizer)
    config.load_model(config.LOAD_CHECKPOINT_FOLDER,target_model,optimizer_target)

#Initialize the Mean Squared Error Loss
#huber_loss=nn.HuberLoss(delta=1.0)
mean_squared_error = torch.nn.MSELoss()

#Define the Action Space, that is composed by Actions (Steering Wheel, Throttle, Break)
action_space = [
                (-1, 1, 0.2), (0, 1, 0.2), (1, 1, 0.2),
                (-1, 1,   0), (0, 1,   0), (1, 1,   0),
                (-1, 0, 0.2), (0, 0, 0.2), (1, 0, 0.2),
                (-1, 0,   0), (0, 0,   0), (1, 0,   0)
              ]


#Here it follow the definition of the Functions that contain the Policy to choose the Action to perform

#Q-Table Policy
def select_action(state, epsilon):
    rv = random.uniform(0, 1)
    if rv < epsilon:
        return action_space[random.randrange(len(action_space))]          #We sample a random action
        #return env.action_space.sample()
    else:
        if state in Q_Table:
            return np.argmax(Q_Table[state])
        else:
            return (0, 1,   0)

#Neural Network Policy
def select_action_nn(state, epsilon):
    rv = random.uniform(0, 1)
    if rv < epsilon:
        return action_space[random.randrange(len(action_space))]          #We sample a random action

    else:
        prediction = model(torch.from_numpy(state.astype('float32')).to(config.DEVICE)).detach().cpu().numpy()
        action = action_space[np.argmax(prediction)]                      #Select the action with the maximum predicted Q-Value
        return action


#Here it follow the definition of the Functions that update the value of Epsilon

#Update Epsilon in each iteration until it converges to MIN_EPSILON
def update_epsilon(epsilon):
    epsilon -= epsilon/100 # reduce epsilon by 1/100
    if epsilon<=config.MIN_EPSILON:
        return config.MIN_EPSILON
    else:
        return epsilon

#Update Epsilon in each episode multiplying it by the Decayì
def update_epsilon_nn(epsilon):
    epsilon *= decay
    if epsilon<=config.MIN_EPSILON:
        return config.MIN_EPSILON
    else:
        return epsilon


def discretize_image(image):
    image = image.unsqueeze(0)
    pooling = nn.AvgPool2d(3, stride=3)
    image = pooling(image)
    image = image.squeeze(0)
    image = torch.round(image, decimals=-1)
    #buff = io.BytesIO()
    #torch.save(image, buff)
    #buff.seek(0)
    #image = buff.getvalue().toString()
    transform = transforms.ToPILImage()
    pil_image = transform(image)
    # Convert the PIL image to bytes
    image_bytes = pil_image.tobytes()
    # Encode the bytes to base64
    base64_string = base64.b64encode(image_bytes).decode()
    return base64_string


#Initialize the Car Racing Environment
env = gym.make("CarRacing-v2", render_mode="human")


if(config.USE_QTABLE):            #Q Table
    #images = []
    #Q_Table = np.zeros([10000000, 12])
    Q_Table = {}

    for i in range(config.NUM_EPISODES):              #Iterate over the Episodes
        state, info = env.reset()               #The state is a 96x96 Matrix

        state = cv2.cvtColor(state, cv2.COLOR_BGR2GRAY)           #Convert the state into a Grayscale Image, that is a Matrix 96x96 composed by Integer values

        state = torch.from_numpy(state.astype('float32'))
        state = discretize_image(state)

        #print(state)

        #imgplot2 = plt.imshow(image)
        #plt.show()

        done = False

        while(True):                    #Iterate over the Steps
            action = select_action(state, epsilon)
            observation, reward, done, truncated, info = env.step(action)
            observation = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)           #Convert the state into a Grayscale Image, that is a Matrix 96x96 composed by Integer values
            image = torch.from_numpy(observation.astype('float32'))
            next_state = discretize_image(image)

            if next_state in Q_Table:
                next_max = np.max(Q_Table[next_state])
            else:
                next_max = 0

            if state in Q_Table:
                #print(Q_Table[state])
                Q_Table[state][action_space.index(action)] += alpha*(reward + config.GAMMA * next_max - Q_Table[state][action_space.index(action)])
            else:
                Q_Table[state] = [0] * 12

            state = next_state

            episode_reward += reward

            if done or truncated:
                break

        print("episode: ", i)
        print("episode cumulative reward : ", episode_reward)
        print("epsilon: ",epsilon)
        epsilon = update_epsilon(epsilon)
        cum_reward_table[i]=episode_reward
        episode_reward = 0 #reset the total reward each episode


    #save the q table for testing
    #np.savetxt('q_table.csv', Q, delimiter=','fmt='%f18')
    #np.savetxt('q_table_little_discretization2000.csv', Q, delimiter=',') # full precision
    np.savetxt('q_table_big_discretization1000.csv', Q_Table, delimiter=',') # full precision

else:             #Use a Neural Network to approximate the Q Function
    for i in range(config.NUM_EPISODES):            #Iterate over the Episodes
        state, info = env.reset()                                 #The state is a 96x96 Matrix
        state = cv2.cvtColor(state, cv2.COLOR_BGR2GRAY)           #Convert the state into a Grayscale Image, that is a Matrix 96x96 composed by Integer values
        #state = state.astype(float)
        #state /= 255.0

        frames_queue = deque([state]*3, maxlen = 3)               #Add the initial state into the Queue

        done = False

        while(True):                    #Iterate over the Steps

            current_frame = np.array(frames_queue)

            action = select_action_nn(current_frame, epsilon)         #Select the aAction with the maximum predicted Q-Value
                                                                      #The Action is composed by 3 Values, that are the steering wheel, gas and breaking

            rew = 0
            #Skip 3 Frames
            for tot in range(3):
                next_state, reward, done, truncated, info = env.step(action)
                rew += reward
                if done or truncated:
                    break

            #If we obtain more than 25 consecutive negative reward, we'll terminate the current episode
            tot_negative_reward = tot_negative_reward + 1 if time_frame_counter > 100 and reward < 0 else 0


            #Increment the Reward if the current Action has the maximum value for the Throttle and the minimum value fot the Break
            if action[1] == 1 and action[2] == 0:
                rew *= 1.5

            episode_reward += rew               #Update the cumulative episode reward

            next_state = cv2.cvtColor(next_state, cv2.COLOR_BGR2GRAY)           #Convert the next state into a Grayscale Image
            #Add normalization?

            frames_queue.append(next_state)                                     #Append the next state to the Queue
            next_frame = np.array(frames_queue)

            #Remove the oldest item if the queue is full, in a way such that we can add a new one
            if len(buffer)>=config.BUFFER_SIZE:
                buffer.popleft()               #We dequeue the oldest item

            buffer.append((current_frame, action_space.index(action), reward, next_frame, done))

            #Stop iterating if the current episode is finished, truncated, it has a negative cumulative reward or it has 25 consecutive negative rewards
            if done or truncated or tot_negative_reward > 25 or episode_reward < 0:
                epsilon = update_epsilon_nn(epsilon)            #Update the value of Epsilon
                print("Current Episode: ", i)
                print("Cumulative reward: ", episode_reward)
                print("Current epsilon: ", epsilon)
                print("########################### \n")
                break

            #Let's train the Neural Network every 4 actions and if the buffer has at least BATCH_SIZE elements
            if(len(buffer) >= config.BATCH_SIZE):
                batch = random.sample(buffer, config.BATCH_SIZE)                #Shuffle randomly the elements of the Buffer

                output_array = []                 #This array will contain the predictions made by the Model with respect to the current Action
                target_array = []                 #This array will contain the Rewards updated summing the prediction made by the Target Model (Using the Bellman's Equation)

                #Iterate over the shuffled elements of the Buffer
                for current_frame, action, reward, next_frame, done in batch:

                    #Use the Target Model to compute the prediction over the next frame
                    next_frame = torch.from_numpy(next_frame.astype('float32')).to(config.DEVICE)
                    next_frame = target_model(next_frame).detach()

                    #Compute the Target using the Bellman's Equation
                    target = reward + (1 - done) * config.GAMMA * max(next_frame)

                    #Append the Target to the Target Array
                    target_array.append(target)

                    #Use the Model to compute the prediction over the current frame
                    current_frame = torch.from_numpy(current_frame.astype('float32')).to(config.DEVICE)
                    output = model(current_frame)

                    #Append the prediction of the current Action to the Output Array
                    output_array.append(output[action])

                #Stack the two Arrays
                output_array = torch.stack(output_array)
                target_array = torch.stack(target_array)

                #Compute the Mean Squared Error Loss
                loss = mean_squared_error(output_array, target_array)

                optimizer.zero_grad()                   #Reset the Gradients
                loss.backward()                         #Execute the Backpropagation of the Loss
                optimizer.step()                        #Update the weights

            time_frame_counter += 1         #Update the counter of the Frames


        #Save the weights of the Network every 5 Episodes
        if (i+1) % 5 == 0:
            config.save_model(model, optimizer, i+1)

            #Save the Weights also on Google Drive
            torch.save({
                "state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(),
            }, "../drive/MyDrive/Checkpoint")

            print("Weigths saved in: "+ config.CHECKPOINT_FOLDER)

        #Update the weights of the Target Network every 5 Episodes
        if (i+1) % 5 == 0:
            config.load_model(config.CHECKPOINT_FOLDER, target_model, optimizer_target)
            print("Target network updated")


        cum_reward_nn[i] = episode_reward                     #Add the current episode's reward to the array

        #Reset the Variables
        episode_reward = 0
        tot_negative_reward = 1
        time_frame_counter = 1


#Close the Environment
env.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
